In [17]:
import keras
from keras.datasets import mnist
import keras.layers as layers
from keras.optimizers import Adam
from keras.models import Model
import numpy as np
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt

In [18]:
# MNISTデータを読込む
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train[:, :, :, np.newaxis]
x_test = x_test[:, :, :, np.newaxis]
# one-hot化
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [19]:
# 入力層
img_input = layers.Input(shape=x_train.shape[1:])

In [20]:
# 第１層
block1_channel = 8
x = layers.Conv2D(block1_channel, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(block1_channel, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

In [21]:
# 第2層
block2_channel = block1_channel  * 2
x = layers.Conv2D(block2_channel, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(block2_channel, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(block2_channel, (3, 3), activation='relu', padding='same', name='block2_conv3')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

In [22]:
# 全結合相
conv_denses = 256
x = layers.Flatten(name='flatten')(x)
x = layers.Dense(conv_denses , activation='relu', name='fc1')(x)
x = layers.Dropout(0.5, name='dropout1')(x)
x = layers.Dense(conv_denses , activation='relu', name='fc2')(x)
x = layers.Dropout(0.5, name='dropout2')(x)
x = layers.Dense(10, activation='softmax', name='predictions')(x)

In [23]:
# モデル構築
model = Model(img_input, x, name='vgg16')
adam = Adam()
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [8]:
# 学習
BATCH_SIZE = 128
EPOCH = 10
history = model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCH,  verbose=1, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 94s 2ms/step - loss: 0.3999 - acc: 0.8767 - val_loss: 0.0656 - val_acc: 0.9786
Epoch 2/10
60000/60000 [==============================] - 93s 2ms/step - loss: 0.1072 - acc: 0.9697 - val_loss: 0.0429 - val_acc: 0.9862
Epoch 3/10
60000/60000 [==============================] - 94s 2ms/step - loss: 0.0780 - acc: 0.9785 - val_loss: 0.0368 - val_acc: 0.9875
Epoch 4/10
60000/60000 [==============================] - 93s 2ms/step - loss: 0.0648 - acc: 0.9817 - val_loss: 0.0418 - val_acc: 0.9872
Epoch 5/10
60000/60000 [==============================] - 92s 2ms/step - loss: 0.0553 - acc: 0.9840 - val_loss: 0.0315 - val_acc: 0.9905
Epoch 6/10
60000/60000 [==============================] - 93s 2ms/step - loss: 0.0491 - acc: 0.9863 - val_loss: 0.0347 - val_acc: 0.9899
Epoch 7/10
60000/60000 [==============================] - 95s 2ms/step - loss: 0.0433 - acc: 0.9877 - val_loss: 0.0322 - val_acc

In [9]:
# モデルと重みの保存
json_string = model.to_json()
open("my_model.json", 'w').write(json_string)
model.save_weights('my_model_weights.h5')

In [24]:
# 検証
score = model.evaluate(x_test, y_test, verbose=1)
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 3s 345us/step

Test loss: 11.46660728149414
Test accuracy: 0.0977


In [25]:
# 間違い見つける
predicts = model.predict(x_test)

In [26]:
number=1
print(predicts[number,:])
print(y_test[number])

[2.5959816e-08 1.2570596e-12 2.9274310e-09 1.0498509e-01 2.4628214e-04
 6.2673655e-12 1.5349119e-08 8.3537827e-10 8.9476866e-01 1.8763105e-08]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
